# Fase 1.3: Optimización del modelo en HAR (Hailo Archive)

In [1]:
# Imports

import json
import os
import gc

import math
import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
import torch
import onnx
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from itertools import islice

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/')
import resbagan_networks
import resbagan_datasets

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

2025-06-08 18:47:18.984130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 18:47:19.758272: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pablo/anaconda3/envs/hailo_gpu_env/lib/python3.8/site-packages/nvidia/dali/backend.py:77: Warning: DALI 1.49 is the last release to support Python 3.8 Please update your environment to use Python 3.9, or newer.
  deprecation_warning(


## 1. Optimización detallada

### 1.1. Carga de datos

In [2]:
# Datos de entrada

batch_size = 100
B = 5
sizex = 32
sizey = 32

In [3]:
# Definir parámetros y cargar datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'
MODEL="../results/models/model_ResBaGAN.pt"

# Proporción de entrenamiento, validación y test
SAMPLES=[0.02,0.01]

# Carga de datos para la inferencia en el discriminador
dataset = resbagan_datasets.HyperDataset(
    "oitaven_river", segmented=False, patch_size=32, ratios=(SAMPLES[0], SAMPLES[1]))

# Almacenamos las dimensiones en variables
H = dataset.height
V = dataset.width

# En este caso seleccionamos samples aleatorios
samples = dataset.test_set['samples']

# Obtenemos el array de gt
truth = dataset.gt.flatten()
# Obtenemos un array de indices para test
test = dataset.test_index_list

# Obtenemos el numero de clases y el numero de clases no vacias para test
nclases = dataset.classes_count
nclases_no_vacias = 0
for i in range(nclases):
    clase_actual = i + 1
    if any(truth[idx] == clase_actual for idx in test):
        nclases_no_vacias += 1

print(dataset)

# The custom HyperDataset object contains all the train, validation and test data
#   --> But it will wrapped into a PyTorch data feeder for convenience
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=100,
    shuffle=True,
    num_workers=4,
    pin_memory=False,
)

[*] Loading dataset oitaven_river from disk
	oitaven_river dataset is in RAW format
[*] Recording available classes
[*] Starting preprocessing
[*] Scaling dataset to [-1, 1]
[*] Splitting dataset into train, validation, and test sets: ratios (0.02, 0.01)
[*] Total samples: 6067179
	[*] Recording samples for class Water (309248 items)
	[*] Recording samples for class Bare soil (113324 items)
	[*] Recording samples for class Rock (79152 items)
	[*] Recording samples for class Asphalt (43861 items)
	[*] Recording samples for class Concrete (128022 items)
	[*] Recording samples for class Tiles (78785 items)
	[*] Recording samples for class Meadows (2428482 items)
	[*] Recording samples for class Native trees (1829360 items)
	[*] Recording samples for class Pines (193884 items)
	[*] Recording samples for class Eucalyptus (863061 items)

[*] HyperDataset summary:
	Name: oitaven_river
	Shape: (height) 6689, (width) 6722, (bands) 5
	Classes: ['Water', 'Bare soil', 'Rock', 'Asphalt', 'Concrete'

In [4]:
# Cargar el modelo convertido a ONNX

ort_session = ort.InferenceSession("../results/models/model_ResBaGAN_discriminator.onnx")

# Cargar el modelo en HAR

model_name = "../results/models/model_ResBaGAN_discriminator"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

### 1.2 Evaluar el modelo en har sin optimización

In [6]:
# Comprobacion rapida de la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = (torch.rand(1, B, sizex, sizey) * 2 - 1).to("cpu")
input_np = input_tensor.cpu().numpy()

# Realizar la inferencia en ONNX
output_onnx = ort_session.run(None, {'input': input_np})[0]

print(output_onnx)

# Realizar la inferencia en HAR
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    input_har = np.transpose(input_np, (0, 2, 3, 1))

    # Realizar la inferencia en el modelo .har
    output_har = runner.infer(ctx, input_har)[0]

    print(output_har) 

# Comparar las diferencias
error = np.abs(output_onnx - output_har).mean()
print(f'Error medio entre ONNX y HAR: {error}')

[[ -4.4338574   1.9349066  -1.1357371 -12.580229   -3.8186932   1.6274607
   -6.9782634  -1.3601333   5.832494   -5.047649    8.855347 ]]
[info] Using 1 GPU for inference


2025-06-07 20:57:12.424312: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-06-07 20:57:12.524259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 8entries [00:00, 247.31entries/s]

[[[[ -2.5148022   1.4130999  -0.5689734 -10.232361   -1.7181966
      0.7633981  -2.620554   -2.6805184   2.9597077  -3.3096926
      5.988242 ]]]]
Error medio entre ONNX y HAR: 1.952363133430481


In [5]:
# Evaluar el modelo en HAR sin optimizacion

# Modo evaluación
dataset.to_test()
data_iter = iter(data_loader)

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    (inputs, labels, targets_pixel_level) = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    outputs_discriminator = outputs[0]
    outputs_discriminator[:, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
    preds_onnx=np.argmax(outputs_discriminator, axis=1)

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)[0]
        native_res[:, 0, 0, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

2025-06-07 23:34:04.413847: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-07 23:34:04.419331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-07 23:34:04.419525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Using 1 GPU for inference


Inference: 0entries [00:00, ?entries/s]2025-06-07 23:34:07.443633: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-06-07 23:34:14.600249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2025-06-07 23:34:15.915506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Inference: 104entries [00:08, 11.81entries/s]

[Iteración 1] Error medio: 0.5300 ; Coincidencias: 73/100



2025-06-07 23:34:16.336811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


...

[Iteración 99] Error medio: 0.4000 ; Coincidencias: 77/100


2025-06-07 23:34:44.569055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


[info] Using 1 GPU for inference


Inference: 0entries [00:00, ?entries/s]2025-06-07 23:34:44.669716: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1505.83entries/s]

[Iteración 100] Error medio: 0.3400 ; Coincidencias: 84/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.354600
Precisión global: 82.37%


### 1.3 Aplicar modificaciones de optimización al modelo y evaluarlo

In [6]:
# Crear un model script para el proceso de optimización

model_script_lines = [
    # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # For multiple input nodes:
    # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n',
    # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n',
    # ...
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(model_script_lines))
runner.optimize_full_precision()

[warning] Model script is empty
[info] Loading model script commands to model_ResBaGAN_discriminator from string
[warning] Model script is empty
[warning] DEPRECATION WARNING: Optimizing in full precision will require calibration data in the near future, to allow more accurate optimization algorithms which require inference on actual data.


In [7]:
# Evaluar el modelo en HAR con optimizacion

# Modo evaluación
dataset.to_test()
data_iter = iter(data_loader)

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    (inputs, labels, targets_pixel_level) = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    outputs_discriminator = outputs[0]
    outputs_discriminator[:, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
    preds_onnx=np.argmax(outputs_discriminator, axis=1)

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_FP_OPTIMIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)[0]
        native_res[:, 0, 0, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

[info] Using 1 GPU for inference


2025-06-07 23:35:07.796378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-06-07 23:35:08.558899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:04, 25.80entries/s]

[Iteración 1] Error medio: 0.3900 ; Coincidencias: 79/100



2025-06-07 23:35:12.695495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


[info] Using 1 GPU for inference


...

[Iteración 99] Error medio: 0.3600 ; Coincidencias: 82/100


2025-06-07 23:35:41.344027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


[info] Using 1 GPU for inference


Inference: 0entries [00:00, ?entries/s]2025-06-07 23:35:41.450568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1530.31entries/s]

[Iteración 100] Error medio: 0.3200 ; Coincidencias: 84/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.355800
Precisión global: 82.34%


### 1.4 Cuantizar el modelo y evaluarlo

In [5]:
# Crear un dataset de calibración
# The original images are being used, just as the input to the SDK_FP_OPTIMIZED emulator
total_images = 1050

dataset.to_train()
calib_dataset = np.zeros((total_images, sizex, sizey, B), dtype = np.float32)

count = 0

# Seleccionamos batches del dataloader para guardarlos en el dataset
for (inputs, labels, targets_pixel_level) in data_loader:
    for img in inputs:
        if count >= total_images:
            break
            
        # Los inputs son de la forma (batch_size, B, sizex, sizey)
        img_np = img.numpy()
        # Trasponemos los inputs a formato (batch_size, sizex, sizey, B)
        img_har = np.transpose(img_np, (1, 2, 0))

        calib_dataset[count] = img_har
        count += 1
        
    if count >= total_images:
        break

In [ ]:
# Cuantizar el modelo con el dataset de calibración

# For calling Optimize, use the short version: runner.optimize(calib_dataset)
# A more general approach is being used here that works also with multiple input nodes.
# The calibration dataset could also be a dictionary with the format:
# {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
hn_layers = runner.get_hn_dict()["layers"]
print("Input layers are: ")
print([layer for layer in hn_layers if hn_layers[layer]["type"] == "input_layer"])  # See available input layer names
calib_dataset_dict = {"model_ResBaGAN_discriminator/input_layer1": calib_dataset}  # In our case there is only one input layer

optimization_level = 4
compression_level = 0
# Mapeamos las proporciones de pesos de 4 bits según el nivel de compresión
compression_ratios = {
    0: 0.0,
    1: 0.2,
    2: 0.4,
    3: 0.6,
    4: 0.8,
    5: 1.0
}
auto_4bit_ratio = compression_ratios.get(compression_level, 0.0)

alls_lines = [
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # Batch size is 8 by default; 2 was used for stability on PCs with low amount of RAM / VRAM
    f"model_optimization_flavor(optimization_level={optimization_level}, compression_level={compression_level}, batch_size=8)\n",
    # The following line is needed because this is a really small model, and the compression_level is always reverted back to 0.'
    # To force using compression_level with small models, the following line should be used (compression level=4 equals to 80% 4-bit):
    f"model_optimization_config(compression_params, auto_4bit_weights_ratio={auto_4bit_ratio})\n",
    # The application of the compression could be seen by the [info] messages: "Assigning 4bit weight to layer .."
]

runner.load_model_script("".join(alls_lines))

runner.optimize(calib_dataset_dict)

Input layers are: 
['model_ResBaGAN_discriminator/input_layer1']
[info] Loading model script commands to model_ResBaGAN_discriminator from string
[info] Starting Model Optimization


2025-06-08 18:48:47.317625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-08 18:48:47.330404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-08 18:48:47.330667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Model received quantization params from the hn


2025-06-08 18:48:50.742883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-08 18:48:50.745101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-08 18:48:50.745344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.20)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


...

Training:  18%|▏| 7514/40960 [00:18<01:20, 413.09batches/s, l2_loss: 0.0427 - round_loss
Training:  18%|▏| 7514/40960 [00:18<01:20, 413.09batches/s, l2_loss: 0.0426 - round_loss
Training:  19%|▏| 7603/40960 [00:18<01:19, 421.80batches/s, l2_loss: 0.0426 - round_loss
Training:  19%|▏| 7603/40960 [00:18<01:19, 421.80batches/s, l2_loss: 0.0428 - round_loss
Training:  19%|▏| 7686/40960 [00:18<01:19, 417.57batches/s, l2_loss: 0.0428 - round_loss
Training:  19%|▏| 7686/40960 [00:18<01:19, 417.57batches/s, l2_loss: 0.0427 - round_loss
Training:  19%|▏| 7772/40960 [00:19<01:18, 420.79batches/s, l2_loss: 0.0427 - round_loss
Training:  19%|▏| 7772/40960 [00:19<01:18, 420.79batches/s, l2_loss: 0.0427 - round_loss
Training:  19%|▏| 7860/40960 [00:19<01:17, 426.20batches/s, l2_loss: 0.0427 - round_loss
Training:  19%|▏| 7860/40960 [00:19<01:17, 426.20batches/s, l2_loss: 0.0426 - round_loss
Training:  19%|▏| 7947/40960 [00:19<01:17, 427.34batches/s, l2_loss: 0.0426 - round_loss
Training:  19%|▏| 794

Training:  27%|▎| 11134/40960 [00:27<01:15, 396.77batches/s, l2_loss: 0.0408 - round_los
Training:  27%|▎| 11216/40960 [00:27<01:14, 400.07batches/s, l2_loss: 0.0408 - round_los
Training:  27%|▎| 11216/40960 [00:27<01:14, 400.07batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11291/40960 [00:27<01:15, 392.44batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11291/40960 [00:27<01:15, 392.44batches/s, l2_loss: 0.0407 - round_los
Training:  28%|▎| 11367/40960 [00:28<01:16, 387.29batches/s, l2_loss: 0.0407 - round_los
Training:  28%|▎| 11367/40960 [00:28<01:16, 387.29batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11443/40960 [00:28<01:16, 385.10batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11443/40960 [00:28<01:16, 385.10batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11519/40960 [00:28<01:16, 382.69batches/s, l2_loss: 0.0409 - round_los
Training:  28%|▎| 11519/40960 [00:28<01:16, 382.69batches/s, l2_loss: 0.0407 - round_los
Training:  28%|▎| 115

Training:  36%|▎| 14657/40960 [00:36<01:08, 385.01batches/s, l2_loss: 0.0404 - round_los
Training:  36%|▎| 14657/40960 [00:36<01:08, 385.01batches/s, l2_loss: 0.0404 - round_los
Training:  36%|▎| 14733/40960 [00:36<01:08, 382.02batches/s, l2_loss: 0.0404 - round_los
Training:  36%|▎| 14733/40960 [00:36<01:08, 382.02batches/s, l2_loss: 0.0406 - round_los
Training:  36%|▎| 14814/40960 [00:36<01:07, 388.29batches/s, l2_loss: 0.0406 - round_los
Training:  36%|▎| 14814/40960 [00:36<01:07, 388.29batches/s, l2_loss: 0.0405 - round_los
Training:  36%|▎| 14896/40960 [00:37<01:06, 393.67batches/s, l2_loss: 0.0405 - round_los
Training:  36%|▎| 14896/40960 [00:37<01:06, 393.67batches/s, l2_loss: 0.0405 - round_los
Training:  37%|▎| 14977/40960 [00:37<01:05, 396.90batches/s, l2_loss: 0.0405 - round_los
Training:  37%|▎| 14977/40960 [00:37<01:05, 396.90batches/s, l2_loss: 0.0405 - round_los
Training:  37%|▎| 15059/40960 [00:37<01:04, 399.58batches/s, l2_loss: 0.0405 - round_los
Training:  37%|▎| 150

Training:  44%|▍| 18187/40960 [00:45<00:57, 395.07batches/s, l2_loss: 0.0403 - round_los
Training:  45%|▍| 18269/40960 [00:45<00:56, 399.31batches/s, l2_loss: 0.0403 - round_los
Training:  45%|▍| 18269/40960 [00:45<00:56, 399.31batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18337/40960 [00:45<00:59, 380.84batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18337/40960 [00:45<00:59, 380.84batches/s, l2_loss: 0.0403 - round_los
Training:  45%|▍| 18409/40960 [00:45<01:00, 373.83batches/s, l2_loss: 0.0403 - round_los
Training:  45%|▍| 18409/40960 [00:45<01:00, 373.83batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18491/40960 [00:46<00:58, 383.52batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18491/40960 [00:46<00:58, 383.52batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18575/40960 [00:46<00:56, 394.09batches/s, l2_loss: 0.0402 - round_los
Training:  45%|▍| 18575/40960 [00:46<00:56, 394.09batches/s, l2_loss: 0.0402 - round_los
Training:  46%|▍| 186

Training:  53%|▌| 21611/40960 [00:54<00:49, 393.05batches/s, l2_loss: 0.0401 - round_los
Training:  53%|▌| 21611/40960 [00:54<00:49, 393.05batches/s, l2_loss: 0.0400 - round_los
Training:  53%|▌| 21694/40960 [00:54<00:48, 398.60batches/s, l2_loss: 0.0400 - round_los
Training:  53%|▌| 21694/40960 [00:54<00:48, 398.60batches/s, l2_loss: 0.0400 - round_los
Training:  53%|▌| 21771/40960 [00:54<00:48, 393.09batches/s, l2_loss: 0.0400 - round_los
Training:  53%|▌| 21771/40960 [00:54<00:48, 393.09batches/s, l2_loss: 0.0401 - round_los
Training:  53%|▌| 21847/40960 [00:54<00:49, 388.24batches/s, l2_loss: 0.0401 - round_los
Training:  53%|▌| 21847/40960 [00:54<00:49, 388.24batches/s, l2_loss: 0.0400 - round_los
Training:  54%|▌| 21925/40960 [00:55<00:48, 388.48batches/s, l2_loss: 0.0400 - round_los
Training:  54%|▌| 21925/40960 [00:55<00:48, 388.48batches/s, l2_loss: 0.0401 - round_los
Training:  54%|▌| 22006/40960 [00:55<00:48, 392.60batches/s, l2_loss: 0.0401 - round_los
Training:  54%|▌| 220

In [10]:
# Evaluar el modelo en HAR cuantizado

# Modo evaluación
dataset.to_test()
data_iter = iter(data_loader)

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    (inputs, labels, targets_pixel_level) = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    outputs_discriminator = outputs[0]
    outputs_discriminator[:, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
    preds_onnx=np.argmax(outputs_discriminator, axis=1)

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_QUANTIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)[0]
        native_res[:, 0, 0, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

[info] Using 1 GPU for inference


2025-06-07 23:36:57.287589: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-06-07 23:36:58.477768: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:12,  8.33entries/s]

[Iteración 1] Error medio: 0.2700 ; Coincidencias: 87/100



2025-06-07 23:37:11.067731: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


[info] Using 1 GPU for inference


...

[Iteración 99] Error medio: 0.3600 ; Coincidencias: 85/100


2025-06-07 23:37:57.289458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error


[info] Using 1 GPU for inference


Inference: 0entries [00:00, ?entries/s]2025-06-07 23:37:57.417114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 597.27entries/s]

[Iteración 100] Error medio: 0.2800 ; Coincidencias: 83/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.364300
Precisión global: 81.66%


In [7]:
# Guardar el modelo cuantizado
# Let's save the runner's state to a Quantized HAR
quantized_model_har_path = f"{model_name}_quantized_model_o{optimization_level}_c{compression_level}.har"
runner.save_har(quantized_model_har_path)

[info] Saved HAR to: /home/pablo/Documentos/Enxeñaría Informática/Cuarto/2º Cuatrimestre/Traballo Fin de Grao/hyper-rpi/results/models/model_ResBaGAN_discriminator_quantized_model_o2_c0.har
